<a href="https://colab.research.google.com/github/Dan-Tan/Classification-and-Regression-Tree/blob/main/Decision_Tree_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = np.load('/content/drive/My Drive/Colab Notebooks/Training Data Testing/testingdata.npy')


In [ ]:
#calculating the purity of data set using gini impurity entropy requires more computation and data set is large
def calc_gini_index(data):
    if len(data) > 0:
        _, freq = np.unique(data[:, 81], return_counts = True)
        if freq.shape == (2,):
            neg, pos =  freq
        else:
            neg, pos = freq, 0
        rows_ = len(data[:, 81])
        prob_pos = pos / rows_
        prob_neg = neg / rows_
        gini_index = 1 - (prob_pos**2 + prob_neg**2)
        return gini_index
    else: 
        gini_index = 0
        return gini_index


In [ ]:
#calculates the information gain after the split is made using gini impurity
def information_gain(parent_data, spltL, spltR):
    gini_spltL = calc_gini_index(spltL)
    gini_spltR = calc_gini_index(spltR)
    info_gain = (calc_gini_index(parent_data) - (calc_gini_index(spltL) * (len(spltL[:, 81])/len(parent_data[:, 81]))+\
                                                 calc_gini_index(spltR) * (len(spltR[:, 81])/len(parent_data[:, 81]))))
    return info_gain




In [ ]:
#getting all the possible splits a node can  make, not evaluating the information gain of making each split
def get_possible_splits(data):

    pos_splts = {}

    for i in range(len(data.T)-1):
        splt_vals = np.empty([1,])

        for e in range(16):
            splt_val = np.array([15 * e])
            splt_vals = np.concatenate((splt_vals, splt_val))

        splt_vals = tuple(splt_vals[1:])
        possible_splts = {i: splt_vals}
        pos_splts.update(possible_splts)

    return pos_splts


In [ ]:
#classifying data after node has required purity
def classify_end_data(data):
    label_col = data[:, 81]
    label, label_counts = np.unique(label_col, return_counts = True)
    ind = label_counts.argmax()
    classification = label[ind] 
    return classification

In [ ]:
def check_purity(data):
    purity = calc_gini_index(data)
    if purity < 0.00001:
        label = classify_end_data(data)
        

In [ ]:
def split_data(data, split_val, col):

    splitL = np.empty([82,])
    splitR = np.empty([82,])

    for i in range(len(data)):
        if data[i, col] <= split_val:
            splitL = np.concatenate((splitL, data[i, :]))
        elif data[i, col] > split_val:
            splitR = np.concatenate((splitR, data[i, :]))
    splitL = splitL[82:]
    splitR = splitR[82:]
    colindL = len(splitL) // 82
    colindR = len(splitR) //82
    if splitL.shape[0] > 0:
        splitL = splitL.reshape([colindL, 82])
    if splitR.shape[0] > 0:
        splitR = splitR.reshape([colindR, 82])
    return splitL, splitR
         



In [ ]:
def find_best_split(data):
    best_info_gain = 0
    split_value = 0
    pos_splits = get_possible_splits(data)
    for i in range(len(data.T)-1):
        for e in range(16):
            split_val = pos_splits[i][e]
            splitL, splitR = split_data(data, split_val, i)
            if len(splitL) > 0 and len(splitR) > 0:
                info_gain = information_gain(data, splitL, splitR)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    col = i
                    fin_split_val = split_val
                    spltLfin = splitL
                    spltRfin = splitR
    

    return col, fin_split_val, spltLfin, spltRfin


col, fin_split_val, _, _ = find_best_split(data)
print(col)
print(fin_split_val)

In [ ]:
#uses dictionaries within dictonary to represent the decision tree

def grow_tree(data):
    print("---------------")
    print(len(data))
    print(calc_gini_index(data))
    print(np.count_nonzero(data[:, 81] > 0))
    purity = calc_gini_index(data)

    if purity < 0.05:
        classification = classify_end_data(data)
        print(classification)
        return classification

    else:
        col, fin_split_val, splitLfin, splitRfin = find_best_split(data)
        splitL, splitR = split_data(data, fin_split_val, col)
        question = "{} <= {}".format(col, fin_split_val)
        sub_tree = {question:[]}

        splitLQ, splitRQ = grow_tree(splitL), grow_tree(splitR)

        sub_tree[question].append(splitLQ)
        sub_tree[question].append(splitRQ)
        return sub_tree

y = grow_tree(data)
print(y)


In [ ]:
f = open("/content/drive/My Drive/Colab Notebooks/Training Data Testing/tree.txt", "w")
f.write(str(y))
f.close()